## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [116]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

In [117]:
from langchain_groq import ChatGroq
model =ChatGroq(model="gemma2-9b-it")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001A69CB44E90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001A69CCB8DD0>, model_name='gemma2-9b-it')

In [118]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content = "Hi, My name is Aswani and i can be a nice Ai Engineer.")])

AIMessage(content="Hi Aswani!  \n\nThat's great to hear!  It sounds like you're passionate about AI engineering.  \n\nWhat are some of the things that excite you most about this field?  \n\nI'm always eager to learn more about what people find interesting about artificial intelligence. 😊\n", response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 25, 'total_tokens': 89, 'completion_time': 0.116363636, 'prompt_time': 0.00141719, 'queue_time': 0.25561158, 'total_time': 0.117780826}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5e3227b4-1135-4fa7-bb7b-3639d89d48b3-0', usage_metadata={'input_tokens': 25, 'output_tokens': 64, 'total_tokens': 89})

In [119]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content = "Hi, My name is Aswani and i a good ai Engineer."),
        AIMessage(content = "Hi Aswani, it's nice to meet you!  \n\nIt's great to hear that you're interested in becoming an AI Engineer. It is a fascinating and rapidly growing field with lots of opportunities to make a real impact. What aspects of AI engineering are you most interested in? Do you have any experience or projects you've worked on already? I'd love to hear more about your journey!\n"),
        HumanMessage(content="Hey What's my name and what do i do ?")
    ]
)

AIMessage(content="You told me your name is Aswani and that you're a good AI Engineer!  😊 \n\nIs there anything else you'd like to tell me about yourself?  \n", response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 131, 'total_tokens': 171, 'completion_time': 0.072727273, 'prompt_time': 0.0033884, 'queue_time': 0.25517598999999996, 'total_time': 0.076115673}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0e4df20e-a4d8-47f4-9e2b-3b7b84910c53-0', usage_metadata={'input_tokens': 131, 'output_tokens': 40, 'total_tokens': 171})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [120]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Defining a function to make sure that one session is completely different
# from other session. its return type is basechatmessagehistory
store = {}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [121]:
config = {"configurable":{"session_id": "chat1"}}

In [122]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Aswani and i am a good ai Engineer.")],
    config=config
)

In [123]:
response.content

"Hello Aswani, it's nice to meet you! That's great to hear that you're a good AI Engineer. \n\nWhat kind of AI work are you interested in? Maybe we can talk about some projects or challenges you're facing.  I'm always eager to learn from experienced AI professionals like yourself.\n"

In [124]:
with_message_history.invoke(
    [HumanMessage(content="What's my name ?")],
    config=config
)

AIMessage(content='Your name is Aswani.  \n\nI remember that from our first interaction. 😊 \n\n', response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 109, 'total_tokens': 131, 'completion_time': 0.04, 'prompt_time': 0.00328246, 'queue_time': 0.25317598, 'total_time': 0.04328246}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-bbf90d29-7ff4-4ba4-b5f4-852dcdeda488-0', usage_metadata={'input_tokens': 109, 'output_tokens': 22, 'total_tokens': 131})

In [125]:
## change the config -->session id
config1={"configurable":{"session_id":"chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="What's my name ?")],
    config=config1
)

response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to know! 😊\n"

In [126]:
# Giving data to config1
with_message_history.invoke(
    [HumanMessage(content="My name is Kaite")],
    config=config1
)

AIMessage(content="Hi Kaite, it's nice to meet you! 😊  \n\nIs there anything I can help you with today?  \n", response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 67, 'total_tokens': 96, 'completion_time': 0.052727273, 'prompt_time': 0.002335281, 'queue_time': 0.25425238899999997, 'total_time': 0.055062554}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-d8bccdfc-1453-4648-b34f-c93bafe57697-0', usage_metadata={'input_tokens': 67, 'output_tokens': 29, 'total_tokens': 96})

In [127]:
## change the config -->session id
config1={"configurable":{"session_id":"chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="What's my name ?")],
    config=config1
)

response.content

'Your name is Kaite.  😊 \n\nI remember! \n\nAnything else I can help you with?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [128]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a best mental health Assistant. Help people to get rid of their mental stress and anxiety."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

In [129]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Aswani.")]})

AIMessage(content="Hi Aswani, it's nice to meet you! I understand you're looking for ways to manage your mental stress and anxiety. While I'm not a therapist or medical professional, I can offer some helpful strategies and resources. \n\nIt's important to remember that you're not alone in feeling this way, and there are many things you can do to feel better.  \n\nTo get started, could you tell me a little more about what's been causing you stress and anxiety?  Understanding your specific situation will help me give you more tailored advice. \n\nIn the meantime, here are a few general tips that might be helpful:\n\n* **Practice mindfulness:** Pay attention to the present moment without judgment. This can help you calm your mind and reduce racing thoughts. There are many mindfulness apps and guided meditations available.\n* **Engage in physical activity:** Exercise has been shown to be effective in reducing stress and anxiety. Find something you enjoy, whether it's walking, running, danc

In [130]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [131]:
config = {"configurable": {"session_id":"chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is aswani")],
    config=config
)

response

AIMessage(content="Hi Aswani, it's nice to meet you. I understand you're looking for ways to manage stress and anxiety. While I can't provide medical advice, I can offer some helpful strategies and resources. \n\nIt's important to remember that you're not alone in this. Many people struggle with stress and anxiety, and there are effective ways to cope.\n\nTo help me understand your situation better, could you tell me a little more about:\n\n* **What are some of the things that cause you stress and anxiety?**\n* **How long have you been experiencing these feelings?**\n* **Have you tried any coping mechanisms in the past?**\n\n\nOnce I have a better understanding of your needs, I can offer more personalized advice.  \n\nIn the meantime, here are some general tips that may be helpful:\n\n* **Practice mindfulness and relaxation techniques:** Deep breathing exercises, meditation, and yoga can help calm your mind and body.\n* **Engage in regular physical activity:** Exercise releases endorph

In [132]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are a best mental health Assistant. Help people to get rid of their mental stress and anxiety in {language}."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

In [133]:
response=chain.invoke({"messages":[HumanMessage(content="Hi my name is aswani")],"language":"Hindi"})
response.content

'Namaste Aswani, \n\nमुझे खुशी है कि आप मेरे पास आये हैं। तनाव और चिंता बहुत आम समस्या है, और आप अकेले नहीं हैं। मैं आपकी मदद करने के लिए यहाँ हूँ। \n\nआपके तनाव और चिंता को दूर करने में मैं कुछ चीजें कर सकता हूँ:\n\n* **शांत और ध्यान करने के व्यायाम:** मैं आपको सरल और प्रभावी ध्यान और गहरी सांस लेने के व्यायाम सिखा सकता हूँ जो तुरंत आपको आराम दिला सकते हैं।\n* **आत्म-चिंतन:** मैं आपको आपके विचारों और भावनाओं को समझने में मदद कर सकता हूँ और उनसे निपटने के स्वस्थ तरीके खोजने में।\n* **तनाव प्रबंधन तकनीकें:** मैं आपको तनावपूर्ण परिस्थितियों से निपटने के लिए उपयोगी उपकरण और रणनीतियाँ सिखा सकता हूँ।\n* **पॉजिटिव सोच:** मैं आपको अपने विचारों को अधिक सकारात्मक और आशावादी बनाने में मदद कर सकता हूँ।\n* **सहायक संसाधन:** मैं आपको मानसिक स्वास्थ्य सेवा प्रदाताओं, समर्थन समूहों और अन्य उपयोगी संसाधनों के बारे में जानकारी प्रदान कर सकता हूँ।\n\nआपको किस तरह की मदद की ज़रूरत है? क्या आप किसी विशेष चिंता या तनाव के बारे में बात करना चाहेंगे? \n\nमुझे बताएं, और मैं आपकी सहायता करने के लिए हर संभव प्र

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [134]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [135]:
config2 = {"configurable": {"session_id":"chat4"}}
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi, i am aswani")],"language":"Hindi"},
    config=config2
)
response.content

'नमस्ते अश्वनी, \n\nमुझे बहुत खुशी है कि आप यहाँ हैं! मैं आपकी मदद करने के लिए यहाँ हूँ। तनाव और चिंता हर किसी के जीवन में आते हैं, लेकिन वे हमारे स्वास्थ्य और कल्याण को प्रभावित कर सकते हैं।  \n\nआपको अपनी चिंता और तनाव के बारे में बताएं, ताकि मैं आपकी स्थिति को बेहतर समझ सकूं और आपको मदद करने के लिए कुछ कारगर सुझाव दे सकूं। \n\nयहाँ कुछ सामान्य तरीके हैं जो तनाव और चिंता को कम करने में मदद कर सकते हैं:\n\n* **ध्यान और गहरी साँस लेने की तकनीकें:** \n   ध्यान और गहरी साँस लेने से आपके मन को शांत करने और तनाव के हार्मोन को कम करने में मदद मिल सकती है। \n\n* **नियमित व्यायाम:** \n   व्यायाम शरीर और मन दोनों को तनाव से राहत दिला सकता है। \n\n* **पर्याप्त नींद:** \n   जब आप पर्याप्त नींद लेते हैं, तो आप तनाव का बेहतर सामना कर सकते हैं। \n\n* **स्वस्थ आहार:** \n   एक संतुलित आहार आपको ऊर्जावान और तनाव के प्रति अधिक प्रतिरोधी बनाता है।\n\n* **समय प्रबंधन:** \n   अपने समय का प्रभावी ढंग से प्रबंधन करना आपको अधिक नियंत्रण महसूस करा सकता है और तनाव को कम करने में मदद कर सकता है। \n\n* **अपने प्

In [136]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name is ?")],"language":"german"},
    config=config2
)
response.content

'Dein Name ist Aswani. 😊 \n\nHast du noch Fragen für mich? \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [137]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer = trim_messages(
    max_tokens = 45,
    strategy = "last",
    token_counter = model,
    include_system = True,
    allow_partial = True,
    start_on = "human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [138]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
)
response=chain.invoke(
    {
        "messages":messages + [HumanMessage(content="What ice cream do i like ?")],
    "language":"English"
    }
)
response.content

"As a helpful AI, I don't have access to your personal information, like your favorite ice cream flavor. 🍦\n\nWhat's YOUR favorite flavor?  Maybe I can tell you some fun facts about it! 😊\n"

In [139]:
response = chain.invoke(
    {
        "messages":messages + [HumanMessage(content="What math problem did i asked to you ?")],
        "language": "English"
    }
)
response.content

'You asked me "What\'s 2 + 2?".  😊  \n\n\nIs there anything else I can help you with? I\'m here to chat and offer support if you need it.\n'

In [140]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config5={"configurable": {"session_id":"chat5"}}


In [141]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name is ?")],
     "language":"English"},
    config=config5
)
response.content

"It's great that you're reaching out for help! \n\nWhile I can offer support and information, remember that I'm not a therapist or medical professional. If you're struggling with mental stress and anxiety, please consider talking to a human counselor or doctor.\n\nTo help you manage stress and anxiety, I can offer some strategies:\n\n* **Deep Breathing:**  Try inhaling slowly for a count of 4, holding for 4, and exhaling for 4. Repeat this several times.\n* **Mindfulness:** Pay attention to the present moment without judgment. Notice your thoughts, feelings, and sensations without getting carried away by them.\n* **Exercise:** Physical activity can release endorphins, which have mood-boosting effects. Aim for at least 30 minutes of moderate-intensity exercise most days of the week.\n* **Healthy Diet:** Eat a balanced diet with plenty of fruits, vegetables, and whole grains. Avoid processed foods, sugary drinks, and excessive caffeine.\n* **Sleep:** Aim for 7-9 hours of quality sleep ea